In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# όρος b για την εξίσωση Poisson ως προς την πίεση
def build_up_b(b, rho, dt, u, v, dx, dy):
    
    b[1:-1, 1:-1] = (rho * (1 / dt * 
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / 
                     (2 * dx) + (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)) -
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx))**2 -
                      2 * ((u[2:, 1:-1] - u[0:-2, 1:-1]) / (2 * dy) *
                           (v[1:-1, 2:] - v[1:-1, 0:-2]) / (2 * dx))-
                          ((v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy))**2))

    return b

# Εξίσωση Poission για τη διόρθωση της πίεσης
def pressure_poisson(p, dx, dy, b):
    pn = numpy.empty_like(p)
    pn = p.copy()
    
    for q in range(nit):
        pn = p.copy()
        p[1:-1, 1:-1] = (((pn[1:-1, 2:] + pn[1:-1, 0:-2]) * dy**2 + 
                          (pn[2:, 1:-1] + pn[0:-2, 1:-1]) * dx**2) /
                          (2 * (dx**2 + dy**2)) -
                          dx**2 * dy**2 / (2 * (dx**2 + dy**2)) * 
                          b[1:-1,1:-1])

        p[:, -1] = p[:, -2] # dp/dx = 0 at x = 2
        p[0, :] = p[1, :]   # dp/dy = 0 at y = 0
        p[:, 0] = p[:, 1]   # dp/dx = 0 at x = 0
        p[-1, :] = 0        # p = 0 at y = 2
        
    return p

# Επίλυση των εξισώσεων Navier-Stokes με τη μέθοδο της διόρθωσης της πίεσης
def cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu):
    un = numpy.empty_like(u)
    vn = numpy.empty_like(v)
    b = numpy.zeros((ny, nx))
    
    # Επανάληψη για τον αριθμό των χρονικών βημάτων
    for n in range(nt):
        un = u.copy()
        vn = v.copy()
        
        b = build_up_b(b, rho, dt, u, v, dx, dy)
        p = pressure_poisson(p, dx, dy, b)
        
        # x-ορμής (u)
        u[1:-1, 1:-1] = (un[1:-1, 1:-1]-
                         un[1:-1, 1:-1] * dt / dx *
                        (un[1:-1, 1:-1] - un[1:-1, 0:-2]) -
                         vn[1:-1, 1:-1] * dt / dy *
                        (un[1:-1, 1:-1] - un[0:-2, 1:-1]) -
                         dt / (2 * rho * dx) * (p[1:-1, 2:] - p[1:-1, 0:-2]) +
                         nu * (dt / dx**2 *
                        (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) +
                         dt / dy**2 *
                        (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])))

        # y-ορμής (v)
        v[1:-1,1:-1] = (vn[1:-1, 1:-1] -
                        un[1:-1, 1:-1] * dt / dx *
                       (vn[1:-1, 1:-1] - vn[1:-1, 0:-2]) -
                        vn[1:-1, 1:-1] * dt / dy *
                       (vn[1:-1, 1:-1] - vn[0:-2, 1:-1]) -
                        dt / (2 * rho * dy) * (p[2:, 1:-1] - p[0:-2, 1:-1]) +
                        nu * (dt / dx**2 *
                       (vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 0:-2]) +
                        dt / dy**2 *
                       (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[0:-2, 1:-1])))

        # Συνοριακές συνθηκών
        u[0, :]  = 0
        u[:, 0]  = 0
        u[:, -1] = 0
        u[-1, :] = 1 
        v[0, :]  = 0
        v[-1, :] = 0
        v[:, 0]  = 0
        v[:, -1] = 0
        
        
    return u, v, p

In [ ]:
# Παράμετροι
nx = 41 # Αριθμός κόμβων στον x-άξονα
ny = 41 # Αριθμός κόμβων στον y-άξονα
nt = 1000 # Αριθμός χρονικών βημάτων
nit = 50 # Αριθμός επαναλήψεων για την επίλυση της εξίσωσης Poisson
# c = 1 

Lx = 2 # Μήκος κοιλότητας
Ly = 2 # Ύψος κοιλότητας

dx = Lx / (nx - 1) # Βήμα στον x-άξονα
dy = Ly / (ny - 1) # Βήμα στον y-άξονα
x = numpy.linspace(0, Lx, nx) 
y = numpy.linspace(0, Ly, ny)


X, Y = numpy.meshgrid(x, y) # Πλέγμα


rho = 1 # Πυκνότητα
nu = .1 # Ιξώδες
dt = .001 # Βήμα χρόνου

# Αρχικοποίηση των ταχυτήτων και της πίεσης
u = numpy.zeros((ny, nx))
v = numpy.zeros((ny, nx))
p = numpy.zeros((ny, nx)) 
b = numpy.zeros((ny, nx))

# Επίλυση των εξισώσεων Navier-Stokes
u, v, p = cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu)

In [ ]:



fig = plt.figure(figsize=(11,7), dpi=100)
# plotting the pressure field as a contour
plt.contourf(X, Y, p, alpha=0.5, cmap=cm.viridis)  
plt.colorbar()
# plotting the pressure field outlines
plt.contour(X, Y, p, cmap=cm.viridis)  
# plotting velocity field
plt.quiver(X[::2, ::2], Y[::2, ::2], u[::2, ::2], v[::2, ::2]) 
plt.xlabel('X')
plt.ylabel('Y');

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
Lx = 1.0  # Length of the channel in x-direction
Ly = 1.0  # Length of the channel in y-direction
Nx = 50   # Number of grid points in x-direction
Ny = 50   # Number of grid points in y-direction
dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)
nu = 0.01  # Kinematic viscosity
rho = 1.0  # Density

# Initialize velocity and pressure fields
u = np.zeros((Nx, Ny))
v = np.zeros((Nx, Ny))
p = np.zeros((Nx, Ny))

# Boundary conditions (no-slip at walls)
u[:, 0] = 0
u[:, -1] = 0
v[0, :] = 0
v[-1, :] = 0

# Iteration parameters
max_iter = 1000
tolerance = 1e-6

# Main solver loop
for iteration in range(max_iter):
    u_old = u.copy()
    v_old = v.copy()

    # Update velocity using finite differences
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            u[i, j] = (u_old[i, j] +
                       nu * (u_old[i + 1, j] - 2 * u_old[i, j] + u_old[i - 1, j]) / dx**2 +
                       nu * (u_old[i, j + 1] - 2 * u_old[i, j] + u_old[i, j - 1]) / dy**2)

            v[i, j] = (v_old[i, j] +
                       nu * (v_old[i + 1, j] - 2 * v_old[i, j] + v_old[i - 1, j]) / dx**2 +
                       nu * (v_old[i, j + 1] - 2 * v_old[i, j] + v_old[i, j - 1]) / dy**2)

    # Solve the pressure Poisson equation
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            p[i, j] = ((p[i + 1, j] + p[i - 1, j]) * dy**2 +
                       (p[i, j + 1] + p[i, j - 1]) * dx**2) / (2 * (dx**2 + dy**2)) - \
                      rho * ((u[i + 1, j] - u[i - 1, j]) / (2 * dx) +
                             (v[i, j + 1] - v[i, j - 1]) / (2 * dy))

    # Check for convergence
    if np.max(np.abs(u - u_old)) < tolerance and np.max(np.abs(v - v_old)) < tolerance:
        break

# Plot velocity field or save data as needed
# (You can use matplotlib or any other plotting library)

print(f"Converged after {iteration} iterations.")
